In [1]:
import os
import time
import urllib

import cv2
import numpy as np
import torch
from IPython.display import clear_output, display
from PIL import Image

from sam2.build_sam import build_sam2_object_tracker

In [2]:
class Visualizer:
    def __init__(self,
                 video_width,
                 video_height,
                 save_video=False,
                 output_path='tracked_video.mp4'
                 ):
        
        self.video_width = video_width
        self.video_height = video_height
        self.save_video = save_video
        self.output_path = output_path

        if self.save_video:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            self.video_writer = cv2.VideoWriter(self.output_path, fourcc, 30,
                                                (self.video_width, self.video_height))
        else:
            self.video_writer = None

    def resize_mask(self, mask):
        mask = torch.tensor(mask, device='cpu')
        mask = torch.nn.functional.interpolate(mask,
                                               size=(self.video_height, self.video_width),
                                               mode="bilinear",
                                               align_corners=False,
                                               )

        return mask

    def add_frame(self, frame, mask):
        frame = frame.copy()
        frame = cv2.resize(frame, (self.video_width, self.video_height))
        
        mask = self.resize_mask(mask=mask)
        mask = (mask > 0.0).numpy()
        
        for i in range(mask.shape[0]):
            obj_mask = mask[i, 0, :, :]
            frame[obj_mask] = [255, 105, 180]
                
        self.display_frame_in_notebook(frame)

    def display_frame_in_notebook(self, frame):
        rgb_frame = Image.fromarray(frame)
        clear_output(wait=True)
        display(rgb_frame)

    def stop(self):
        if self.video_writer is not None:
            self.video_writer.release()

In [3]:
# Download Example Video
VIDEO_STREAM = "./TUD-Stadtmitte-raw.webm"
url = 'https://motchallenge.net/sequenceVideos/TUD-Stadtmitte-raw.webm'

# Check if the file already exists
if not os.path.exists(VIDEO_STREAM):
    # If the file doesn't exist, download it
    urllib.request.urlretrieve(url, VIDEO_STREAM)
    print(f"Downloading {VIDEO_STREAM}...")
else:
    print(f"File {VIDEO_STREAM} already exists, skipping download.")

File ./TUD-Stadtmitte-raw.webm already exists, skipping download.


In [4]:
# Set SAM2 Configuration
NUM_OBJECTS = 2
YOLO_CHECKPOINT_FILEPATH = "yolov8x-seg.pt"
SAM_CHECKPOINT_FILEPATH = "../checkpoints/sam2.1_hiera_base_plus.pt"
SAM_CONFIG_FILEPATH = "./configs/samurai/sam2.1_hiera_b+.yaml"
OUTPUT_PATH = VIDEO_STREAM + "_segmented.mp4"
DEVICE = 'cuda:0'

In [5]:
# Open Video Stream
video_stream = cv2.VideoCapture(VIDEO_STREAM)

video_height = int(video_stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_width = int(video_stream.get(cv2.CAP_PROP_FRAME_WIDTH))

# For real-time visualization
visualizer = Visualizer(video_width=video_width,
                        video_height=video_height,
                        save_video=True,
                        output_path=OUTPUT_PATH
                        )

In [6]:
sam = build_sam2_object_tracker(num_objects=NUM_OBJECTS,
                                config_file=SAM_CONFIG_FILEPATH,
                                ckpt_path=SAM_CHECKPOINT_FILEPATH,
                                device=DEVICE,
                                verbose=False
                                )

In [11]:
available_slots = np.inf

first_frame = True
with torch.inference_mode(), torch.autocast('cuda:0', dtype=torch.bfloat16):
    while video_stream.isOpened():
        start_time = time.time()

        # Get next frame
        ret, frame = video_stream.read()
        
        # Exit if no frames remaining
        if not ret:
            break

        # Convert frame from BGR to RGB
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Simulate detection on first frame
        if first_frame:
            
            cv2.imwrite('output_image.jpg', img)
            bbox = np.array([[[172, 92], [254, 325]], 
                             [[334, 76],  [439, 329]]]
                            )
            
            sam_out = sam.track_new_object(img=img,
                                           box=bbox
                                           )
            
            first_frame = False
            
        else:
            sam_out = sam.track_all_objects(img=img)
            
        visualizer.add_frame(frame=frame, mask=sam_out['pred_masks'])

visualizer.stop()
video_stream.release()